### Requirements

In [1]:
pip install --upgrade huggingface_hub

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00


In [4]:
pip install transformers

In [5]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.0 MB/s eta 0:00:00


In [6]:
!pip3 install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00a 0:00:01


In [7]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00 0:00:01


### Import

In [8]:
import pandas as pd

In [9]:
import numpy as np

In [10]:
import torch

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

### Loading Reference Data

In [13]:
df_one = pd.read_csv("KnowledgeTransfers-one_sent.csv")

In [14]:
### Selecting all annotated cases of family relations
df_fam = df[df.relation.isin(["parent_of", "child_of", "siblings", "spouses"])].copy(deep=True)

In [15]:
df_fam.drop(columns=['annotationspan', 'annotation', 'source', 'attributes'], inplace=True)

In [16]:
df_know = df_fam.reset_index().copy(deep=True)

### Preparing Data Set

In [17]:
prop = []
entail = []
for index, row in df_fam.iterrows():
    text = row["relation"]
    person_1 = row["arg1"]
    person_2 = row["arg2"]
    if index % 2 == 0:
        if "parent_of" in text:
            sen = f"{person_1} is the parent of {person_2}."
            prop.append(sen)
            entail.append("entail")
        elif "child_of" in text:
            sen = f"{person_1} is the child of {person_2}."
            prop.append(sen)
            entail.append("entail")
        elif "siblings" in text:
            sen = f"{person_1} and {person_2} are siblings."
            prop.append(sen)
            entail.append("entail")
        elif "spouses" in text:
            sen = f"{person_1} and {person_2} are spouses."
            prop.append(sen)
            entail.append("entail")
    else:
        if "parent_of" in text:
            sen = f"{person_1} is not the parent of {person_2}."
            prop.append(sen)
            entail.append("not entail")
        elif "child_of" in text:
            sen = f"{person_1} is not the child of {person_2}."
            prop.append(sen)
            entail.append("not entail")
        elif "siblings" in text:
            sen = f"{person_1} and {person_2} are spouses."
            prop.append(sen)
            entail.append("not entail")
        elif "spouses" in text:
            sen = f"{person_1} and {person_2} are siblings."
            prop.append(sen)
            entail.append("not entail")

In [18]:
props = pd.Series(prop)
entails = pd.Series (entail)

In [19]:
df_know['proposition'] = props
df_know['classification'] = entails

In [22]:
df_know.classification.value_counts()

entail        50
not entail    39
Name: classification, dtype: int64

In [23]:
df_test = df_know.head().copy(deep=True)

### Loading Model

In [24]:
quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  #bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type = "nf4",
  bnb_4bit_compute_dtype = torch.float16
)

In [ ]:
model_id_1 = "lmsys/vicuna-33b-v1.3"

tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-33b-v1.3", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-33b-v1.3", device_map="auto", quantization_config=quantization_config)

#tokenizer_1 = AutoTokenizer.from_pretrained(model_id_1, use_auth_token=True, use_fast=True)
#model_7b = AutoModelForCausalLM.from_pretrained(model_id_1, use_auth_token=True, device_map="auto", quantization_config=quantization_config)

#hf_7b = pipeline("text-generation", model=model_7b, tokenizer=tokenizer_1, temperature = 0.001)
vicuna = pipeline("text-generation", model="lmsys/vicuna-33b-v1.3")

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/9.87G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/5.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [25]:
model_id_2 = "meta-llama/Llama-2-13b-chat-hf"

tokenizer_2 = AutoTokenizer.from_pretrained(model_id_2, use_auth_token=True, use_fast=True)
model_13b = AutoModelForCausalLM.from_pretrained(model_id_2, use_auth_token=True, device_map="auto", quantization_config=quantization_config)

hf_13b = pipeline("text-generation", model=model_13b, tokenizer=tokenizer_2, temperature = 0.001)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### Creating Prompt Templates

In [26]:
def generate(prompt, max_length=1024, pipe=hf_13b, **kwargs):
    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

In [51]:
results = []
for index, row in df_know.iterrows():
  proposition = row['proposition']
  text = row['context']
  prompt = f"""
  <s>[INST]\n

  Consider the following two texts:

  1. German text: {text}
  2. {proposition}

  Can you determine whether the second proposition {proposition} is entailed by the German text {text}?

  Please provide your answer in the form of a logical statement:
  a.) Yes, the proposition is entailed by the given text.
  b.) No, the proposition is not entailed by the given text.
  Your answer:
  [/INST]
  """
  res = generate(prompt)
  print(res)
  results.append(res)

Based on the given German text JERONIMUS. Weiß nur so viel, daß sein Namen Johann, und er ein unecht Kind des Rupert, – Daß er den Ritterdienst in Rossitz lernte, Und gestern früh das Schwert empfangen hat. SYLVESTER., we can deduce the following logical statement:

a.) Yes, the proposition "johann is the child of rupert" is entailed by the given text.

Explanation:

The text states that Jeronimus (Johann) is the child of Rupert, and that he learned the ritterdienst (knight's service) in Rossitz. These statements imply that Johann is the child of Rupert, as the text explicitly mentions the relationship between the two. Therefore, the proposition "johann is the child of rupert" is entailed by the given text.
Based on the given German text Artig genug, nur zu artig. Aber was soll er deinen Sohn lehren? MAJOR., we cannot determine whether the second proposition CHILD[MAJOR] is the child of major. is entailed.

The German text only provides information about the father's artistic abilities

In [ ]:
results = []
for index, row in df_know.iterrows():
  proposition = row['proposition']
  text = row['context']
  prompt = f"""
  <s>[INST]\n

  A text T textually entails a proposition P, iff typically, a human would be justified in reasoning from the propositions expressed by T to the proposition expressed by H.

  Is the proposition {proposition} entailed by the following piece of German text: {text}?
  Answer with:
  a.) Yes, the proposition is entailed by the given text.
  b.) No, the proposition is not entailed by the given text.
  Your answer:
  [/INST]
  """
  res = generate(prompt)
  print(res)
  results.append(res)

In [52]:
pred = []
for i in results:
    if "Yes, the proposition" in i:
        pred.append("entail")
    else:
        pred.append("not entail")

In [53]:
results_se = pd.Series(pred)

In [54]:
results_se.value_counts()

not entail    76
entail        13
dtype: int64

### Runing Prompts on Two-Sentences

In [55]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [56]:
ground_truth = df_know.classification
predictions = results_se

In [57]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="macro")],
                      [recall_score(ground_truth, predictions,  average="macro")],
                      [precision_score(ground_truth, predictions, average="macro")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.398068,0.493077,0.486336,0.449438
